In [43]:
!conda install -yc conda-forge ipython-sql

Solving environment: ...working... done

# All requested packages already installed.



# INTRODUCTION

In [ ]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

## Overview of the Data
We'll begin by getting a sense of what the data looks like.

In [ ]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

Here are the descriptions for some of the columns:
- `name` — the name of the country.
- `area`— the country's total area (both land and water).
- `area_land` — the country's land area in [square kilometers](https://www.math.net/square-kilometers)
- `area_water` — the country's waterarea in square kilometers.
- `population` — the country's population.
- `population_growth` — the country's population growth as a percentage.
- `birth_rate` — the country's birth rate, or the number of births per year per 1,000 people.
- `death_rate` — the country's death rate, or the number of death per year per 1,000 people.

Let's start by calculating some summary statistics and look for any outlier countries.

## SUMMARY STATISTICS

In [ ]:
%%sql
SELECT MIN(population) AS min_pop, MAX(population) AS max_pop, MIN(population_growth) AS min_pop_growth, MAX(population_growth) AS max_pop_growth
  FROM facts

We see a few interesting things in the summary statistics on the previous screen:

- There's a country with a population of `0`
- There's a country with a population of `7256490011` (or more than 7.2 billion people)

Let's use subqueries to zoom in on just these countries *without* using the specific values.

#### 1. The country with the minimum population

In [ ]:
%%sql
SELECT * 
  FROM facts
 ORDER BY population ASC
 LIMIT 1;

#### 2. The country with the maximum population

In [ ]:
%%sql
SELECT * 
  FROM facts
 ORDER BY population DESC
 LIMIT 1;

It seems like the table contains a row for the whole world, which explains the population of over 7.2 billion. It also seems like the table contains a row for Antarctica, which explains the population of 0.

Now that we know this, we should recalculate the summary statistics we calculated earlier — this time excluding the row for the whole world.

In [ ]:
%%sql
SELECT MIN(population) AS min_pop, MAX(population) AS max_pop, MIN(population_growth) AS min_pop_growth, MAX(population_growth) AS max_pop_growth
  FROM facts
 WHERE name != 'World'

The average value for population and area column

In [ ]:
%%sql
SELECT AVG(population), AVG(area)
  FROM facts
 WHERE name <> 'World'

To finish, we'll find countries that are densely populated. We'll identify countries that have the following:

- Above-average values for population.
- Below-average values for area.

In [ ]:
%%sql
SELECT * 
  FROM facts
 WHERE population > (SELECT AVG(population) 
                     FROM facts
                    WHERE name <> 'World'
                    ) 
     AND area < (SELECT AVG(area) 
                 FROM facts
                WHERE name <> 'World'
                ) 

Some of these countries are generally known to be densely populated, so we have confidence in our results!